## Import Libraries

In [1]:
import pandas as pd
import pickle
import re
import numpy as np
import os

## Read CSV files and merge

In [2]:
pd.options.mode.chained_assignment = None
raw_dataset_dir = "../Raw_Data/"
house_dataset = []
files = [file for file in os.listdir(raw_dataset_dir) if 'csv' in file]
for idx in range(len(files)):
    house_dataset.append(pd.read_csv(raw_dataset_dir + files[idx], index_col = [0]))
dataset = pd.concat(house_dataset, ignore_index=True)
print(dataset.shape)
dataset.head()

(10020, 3)


,Location,Price,Area
0,"380 Phố Tôn Đức Thắng, Phường Hòa Minh, Liên C...",2 triệu/tháng,35 m2
1,"346 Đường Ngũ Hành Sơn, Phường Mỹ An, Ngũ Hành...",2.5 triệu/tháng,35 m2
2,"582 Đường số 2/9, Phường Hòa Cường Nam, Hải Ch...",2 triệu/tháng,35 m2
3,"559 Trần Cao Vân Đường Trần Cao Vân, Phường Xu...",1.7 triệu/tháng,20 m2
4,"Phường Hòa Cường Bắc, Hải Châu, Đà Nẵng",2.2 triệu/tháng,18 m2


## Remove extraordinary data and Modify data

In [3]:
for id in range(len(dataset)):
    dataset['Area'][id] = dataset['Area'][id].split(": ")[-1].replace(',', '.')
    try:
        dataset['Location'][id] = dataset['Location'][id].replace('Phường ', '').replace('P. ', '').replace('Quận ', '').replace('Q. ', '').replace('  ', ' ')
    except: pass
    if '₫' in dataset.Price[idx]:
        if 'Triệu' in dataset.Price[idx]:
            dataset.Price[idx] = dataset.Price[idx].replace('K', '').replace('₫ ', '')
            components = dataset.Price[idx].split(' ')
            if len(components) == 5:
                dataset.Price[idx] = components[0] + '.' + components[2].replace('0', '') + ' ' + components[1] + components[3] + components[4]
            else:
                dataset.Price[idx] = components[0] + ' ' + components[1] + components[2] + components[3]
        else:
            dataset.Price[idx] = dataset.Price[idx].replace('₫', 'đ')
    if(('PN' in dataset['Area'][id]) or ('Tỷ' in dataset['Price'][id]) or ('tỷ' in dataset['Price'][id]) or ('Thương lượng' in dataset['Price'][id]) or ('Thỏa thuận' in dataset['Price'][id])):
        dataset = dataset.drop(index=id)
    
dataset = dataset.reset_index(drop = True)
dataset

,Location,Price,Area
0,"380 Phố Tôn Đức Thắng, Hòa Minh, Liên Chiểu, Đ...",2 triệu/tháng,35 m2
1,"346 Đường Ngũ Hành Sơn, Mỹ An, Ngũ Hành Sơn, Đ...",2.5 triệu/tháng,35 m2
2,"582 Đường số 2/9, Hòa Cường Nam, Hải Châu, Đà ...",2 triệu/tháng,35 m2
3,"559 Trần Cao Vân Đường Trần Cao Vân, Xuân Hà, ...",1.7 triệu/tháng,20 m2
4,"Hòa Cường Bắc, Hải Châu, Đà Nẵng",2.2 triệu/tháng,18 m2
...,...,...,...
9803,"Cho thuê phòng trọ đường Trần Cao Vân, Thanh K...","2,5 triệu",16 m2
9804,"Cho thuê trọ đường Nguyễn Thái Bình, Hòa Minh,...","2,8 triệu",30 m2
9805,"Cho thuê trọ đường Phạm Văn Tráng, Hòa Khánh B...",2 triệu,100 m2
9806,"Cho thuê phòng trọ đường Núi Thành, Hòa Cường ...","1,05 triệu",35 m2


## Extract useful information

### Extract Ward and District in Location 

In [4]:
Locations = dict() 
Locations['Hải Châu'] = ['Bình Hiên', 'Bình Thuận', 'Hải Châu I', 'Hải Châu II', 'Hòa Cường Bắc', 'Hòa Cường Nam', 'Hòa Thuận Đông', 'Hòa Thuận Tây', 'Nam Dương', 'Phước Ninh', 'Thạch Thang', 'Thanh Bình', 'Thuận Phước'] 
Locations['Cẩm Lệ'] = ['Hòa An', 'Hòa Phát', 'Hòa Thọ Đông', 'Hòa Thọ Tây', 'Hòa Xuân', 'Khuê Trung']
Locations['Liên Chiểu'] = ['Hòa Hiệp Bắc', 'Hòa Hiệp Nam', 'Hòa Khánh Bắc', 'Hòa Khánh Nam', 'Hòa Minh']
Locations['Ngũ Hành Sơn'] = ['Hòa Hải', 'Hòa Quý', 'Khuê Mỹ', 'Mỹ An']
Locations['Sơn Trà'] = ['An Hải Bắc', 'An Hải Đông', 'An Hải Tây', 'Mân Thái', 'Nại Hiên Đông', 'Phước Mỹ', 'Thọ Quang']
Locations['Hòa Vang'] = ['Hòa Phong', 'Hòa Bắc', 'Hòa Châu', 'Hòa Khương', 'Hòa Liên', 'Hòa Nhơn', 'Hòa Ninh', 'Hòa Phú', 'Hòa Phước', 'Hòa Sơn', 'Hòa Tiến']
Locations['Thanh Khê'] = ['An Khê', 'Chính Gián', 'Hòa Khê', 'Tam Thuận', 'Tân Chính', 'Thạc Gián', 'Thanh Khê Đông', 'Thanh Khê Tây', 'Vĩnh Trung', 'Xuân Hà']
Locations['Hoàng Sa'] = []


dataset['Ward'] = np.nan

for i in range(len(dataset['Location'])):
    location_components = dataset['Location'][i].split(", ")
    for district in Locations.keys():
        for ward in Locations[district]:
            if ward in location_components:    
                dataset['Ward'][i] = ward
                continue

dataset = dataset.reset_index(drop = True).drop(columns='Location').rename(columns={'Ward':'Location'})
dataset

,Price,Area,Location
0,2 triệu/tháng,35 m2,Hòa Minh
1,2.5 triệu/tháng,35 m2,Mỹ An
2,2 triệu/tháng,35 m2,Hòa Cường Nam
3,1.7 triệu/tháng,20 m2,Xuân Hà
4,2.2 triệu/tháng,18 m2,Hòa Cường Bắc
...,...,...,...
9803,"2,5 triệu",16 m2,Thanh Khê Đông
9804,"2,8 triệu",30 m2,Hòa Minh
9805,2 triệu,100 m2,Hòa Khánh Bắc
9806,"1,05 triệu",35 m2,Hòa Cường Nam


### Extract area value in Area 

In [5]:
#Regex for extracting useful information
area_re = "\d+"

for idx in range(len(dataset['Area'])):
    if 'KXĐ' in dataset.Area[idx]:
        dataset['Area'][idx] = np.nan
    else:
        dataset['Area'][idx] = int(re.findall(area_re, dataset['Area'][idx])[0])
    if dataset['Area'][idx] == 0:
        dataset['Area'][idx] = np.nan
dataset

,Price,Area,Location
0,2 triệu/tháng,35,Hòa Minh
1,2.5 triệu/tháng,35,Mỹ An
2,2 triệu/tháng,35,Hòa Cường Nam
3,1.7 triệu/tháng,20,Xuân Hà
4,2.2 triệu/tháng,18,Hòa Cường Bắc
...,...,...,...
9803,"2,5 triệu",16,Thanh Khê Đông
9804,"2,8 triệu",30,Hòa Minh
9805,2 triệu,100,Hòa Khánh Bắc
9806,"1,05 triệu",35,Hòa Cường Nam


### Extract price value in Price

In [6]:
#Regex for extracting useful information
price_re = "([(\d\W)*\d+]+(Triệu|triệu|Đồng|đ|Nghìn)+)?"
price_stn = "(\d+)\W*(\d*) (Triệu|triệu|Đồng|đ|Nghìn)"

m_price = 1000000
dataset['Price'] = dataset['Price'].apply(lambda x: re.match(price_re, x)[0])
for i in range(len(dataset['Price'])):
    try:
        price_components = list(re.findall(price_stn, dataset['Price'][i])[0])
        if (price_components[2] == 'Triệu' or price_components[2] == 'triệu'):
            if (price_components[1] != ''):
                dataset['Price'][i] = int(float(price_components[0]) * m_price + float(str(0.) + price_components[1]) * 10 * m_price)
            else:
                dataset['Price'][i] = int(float(price_components[0]) * m_price)

        else:
            dataset['Price'][i] = int(float(price_components[0]) * 1000)
    except: dataset['Price'][i] = np.nan

dataset

,Price,Area,Location
0,2000000,35,Hòa Minh
1,2500000,35,Mỹ An
2,2000000,35,Hòa Cường Nam
3,1700000,20,Xuân Hà
4,2200000,18,Hòa Cường Bắc
...,...,...,...
9803,2500000,16,Thanh Khê Đông
9804,2800000,30,Hòa Minh
9805,2000000,100,Hòa Khánh Bắc
9806,1050000,35,Hòa Cường Nam


## Save dataset to csv file

In [ ]:
dataset.to_csv('../Dataset/big_dataset.csv')
SmallDS = dataset.sample(n=1000, random_state=55)
SmallDS.to_csv('../Dataset/small_dataset.csv')